In [2]:
########################## IMPORTS ##########################



from tqdm.notebook import tqdm

import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader

import numpy as np
import os
import re as RE
from matplotlib import pyplot as plt

import os

import lightning as L

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch_geometric
import torch_geometric.data as geom_data
import torch_geometric.nn as geom_nn

from lightning.pytorch.callbacks import ModelCheckpoint
from torch import Tensor

import numpy as np

import pandas as pd
from torch.utils.data import Dataset, DataLoader

from scipy.interpolate import griddata
from functools import reduce

L.seed_everything(42)


Seed set to 42


42

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

True
0
NVIDIA GeForce GTX 1050 Ti


In [3]:

# what do we need for one datapoint:
# BL u/u_e
# BL rho
# BL T/T_e[0] = (T_w/T_e)
# Re
# omega = (2\pi f) * (lscl/u_e)
#  M_e

# labelling: sigma (`s)

########################## LOADING AND STORING DATA ##########################



RESET = False
'''
Tpose = True : Use for returning a list of each data type
Tpose = False: Use for printing out the data. 
'''
def ndata_reader(file,tpose=True):
    arrays = []
    labels = []
    looking = 0
    with open(file, 'r') as a:
        for line in a.readlines():
            if looking > 1:
                break
            if len(RE.findall("Parameters used for normalizatons", line)) > 0:
                looking += 1
            if len(RE.findall("#  Mach number       = ", line)) > 0:
                mach = (RE.findall("[0-9]+\.[0-9]+", line))[0]
            if line[:12] == "variables = " and len(labels)==0:
                labels = line[12:].split(',')
                labels = [l.strip() for l in labels]
            if len(RE.findall("[0-9]+\.[0-9]+", line)) > 3: # condition for dataline is usually more than a few numbers in that row
                arrays.append([float(s) for s in line.split()])

    arrays = np.transpose(arrays) if tpose else arrays
    return labels, arrays, mach

def bl_input_reader(file,tpose=True):
    arrays = []
    with open(file, 'r') as a:
        for line in a.readlines():
            arrays.append([float(s) for s in line.split()])
            
    arrays = np.transpose(arrays) if tpose else arrays
    return ["eta", "rho", "u", "v", "T"], arrays

def all_pos(example):
    # Grab one file to determine the positions of everything
    labels, res, mach = ndata_reader(example)
    res = np.array(res)
    x_pos = labels.index('x')
    sigma_pos = labels.index('`s')
    freq_pos = labels.index('f')
    Re_pos = labels.index('Re')
    lscl_pos = labels.index('lscl')
    ue_pos = labels.index('u_e')
    return [sigma_pos, freq_pos, Re_pos, lscl_pos, ue_pos, mach, x_pos]

# DATAPOINT: U, RHO, TWTE, RE, OMEGA, MACH, SIG (LABEL)

# mode = "second_mode" or "first_mode"
def write_data(root, write_file, mode=1):

    mode = "first_mode" if mode==1 else "second_mode" if mode==2 else None
    # Extract the positions of all important features from an example file
    pos = all_pos("data\\constantRe.tar\\constantRe\\fp_M4.0_Tw_300K\\lst\\second_mode\\case_f_00100000\\nfact.dat")
    sigma_pos = pos[0]
    freq_pos = pos[1]
    Re_pos = pos[2]
    lscl_pos = pos[3]
    ue_pos = pos[4]
    mach = pos[5]

    print("Searching through data in " + root)
    mach_dirs = ["\\" + l for l in os.listdir(root)]

    for m in tqdm(mach_dirs): # :1 because only first one
        print("\tLoading " + m)

        batch = []

        # find the bl profile
        _, bl_profile = bl_input_reader(root + m + "\\mkBaseflow\\bl_input.dat", tpose=True)

        # Grab T_w/T_e which is the first value of T
        TwTe = bl_profile[4][0]

        # We also want to figure out what the boundary thickness is. Since the length scale changes for each output in a frequency result,
        # we want to extract the boundary layer thickness that is nondimensionalized by lscl, which is the value of eta at which u converges 
        # to about 1. I will determine this value by identifying the point at which the difference between two eta values is < 1e-9 or so.
        eta = bl_profile[0]
        u_fine = bl_profile[2] # fine-grained u values to get accurate reading on boundary layer thickness
        ind = 0

        while(ind < len(u_fine)-1 and abs(u_fine[ind] - u_fine[ind+1]) > 1e-9): ind += 1
        thickness_eta = eta[ind]

        # Grab ~60 equidistant points of u and rho information from the boundary layer itself. We only care about the boundary layer so
        # ignore all points after 'ind'
        
        # to get exactly 60 points each time, first find how much buffer we need from the number of points on the boundary layer to get a 
        # number of points divisible by 60
        diff = (ind+1)%60
        # find where the new ind must be set; this number must be divisible by 60
        new_ind = ind + diff + 1 if (ind+diff+1)%60 == 0 else ind + 1 - diff
        step = round(new_ind/60) # just in case the boundary layer has less than 1 point?
        u = bl_profile[2][:new_ind:step]
        rho = bl_profile[1][:new_ind:step]
        
        # now find all lst data from the corresponding mode
        m_lst_mode = m + "\\lst\\" + mode
        lst_2_dirs = ["\\" + l for l in os.listdir(root+m_lst_mode) if len(RE.findall("^case_f_[0-9]+$", l)) > 0]
        for ldir in lst_2_dirs: # :1 because only first one
            # now look for the specific nfact.dat file in this file
            #print("\t\tLoading " + root+m_lst_mode+ldir)
            _, ndata, mach = ndata_reader(root+m_lst_mode+ldir+"\\nfact.dat")
            for line in range(len(ndata[0]) if len(ndata)>0 else 0):
                
                Re_l = ndata[Re_pos][line]
                sig = ndata[sigma_pos][line]
                freq = ndata[freq_pos][line]
                lscl = ndata[lscl_pos][line]
                u_e = ndata[ue_pos][line]

                # Require a dimensional value for boundary layer thickness. To do this, we take the nondim eta value, eta = y/lscl,
                # multiply it with lscl at this point, and have gotten a y value which defines the dim boundary layer thickness 
                thickness = thickness_eta * lscl

                # Now we re-dim Re_l and dim omega with boundary layer thickness 

                omega = 2*np.pi*freq*thickness/u_e
                Re = Re_l * thickness_eta # because thickness_eta = y/lscl, so it divides out the lscl and mults with the y val
                sig = sig * thickness_eta # thickness_eta = y/lscl, so sig * lscl * y/lscl = sig * y
                
                datapoint = []
                datapoint += list(u) 
                datapoint += list(rho) 
                datapoint += [TwTe, Re, omega, mach, sig]
                #print(datapoint)
                #print(len(datapoint))
                # should be 60 + 60 + 5
                
                # add this to the batch
                batch += [datapoint]


        # convert the batch to a dataframe
        batch = np.array(batch)
        df = pd.DataFrame(batch)
        df.to_csv(write_file, mode='a', index=False, header=False)



root = "data\\constantRe.tar\\constantRe"
write_file_root = "data\\csv_files\\"

first_file = write_file_root + "small_first.csv"
sec_file = write_file_root + "small_second.csv"
# reset the csv file
if(RESET):
    if(os.path.isfile(first_file)):
        os.remove(first_file)
    write_data(root, first_file, mode=1) # write the data to the csv file

    if(os.path.isfile(sec_file)):
        os.remove(sec_file)
    write_data(root, sec_file, mode=2)

<>:31: SyntaxWarning: invalid escape sequence '\.'
<>:35: SyntaxWarning: invalid escape sequence '\.'
<>:31: SyntaxWarning: invalid escape sequence '\.'
<>:35: SyntaxWarning: invalid escape sequence '\.'
C:\Users\fouad\AppData\Local\Temp\ipykernel_25992\1873567308.py:31: SyntaxWarning: invalid escape sequence '\.'
  mach = (RE.findall("[0-9]+\.[0-9]+", line))[0]
C:\Users\fouad\AppData\Local\Temp\ipykernel_25992\1873567308.py:35: SyntaxWarning: invalid escape sequence '\.'
  if len(RE.findall("[0-9]+\.[0-9]+", line)) > 3: # condition for dataline is usually more than a few numbers in that row


In [11]:
########################## CREATING DATASETS AND DATALOADERS ##########################

from sklearn.preprocessing import StandardScaler
scaler = [StandardScaler(), StandardScaler()]
sig_scaler = [StandardScaler(), StandardScaler()]
class ProfileDataset(Dataset):

    def __init__(self, file):
        self.data = pd.read_csv(file, header=None)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        res = self.data.iloc[idx]
        res = np.array(res)
        return res

BATCH_SIZE = 256

crazy = False

pfd = []
train_pfd = []
val_pfd = []
test_pfd = []

train_loader = []
val_loader = []
test_loader = []

crazy_loader = []
dum = []
for ind in range(2):
    pfd += [ProfileDataset(first_file if ind==0 else sec_file)]
    dum += [sig_scaler[ind].fit_transform(pfd[ind][:,124:])]
    pfd[ind] = scaler[ind].fit_transform(pfd[ind])

    splits = [int(pfd[ind].__len__() * 0.8), int(pfd[ind].__len__()*0.9)] # 80 train, 10 val, 10 test

    train_pfd += [pfd[ind][:splits[0]]]
    val_pfd += [pfd[ind][splits[0]:splits[1]]]
    test_pfd += [pfd[ind][splits[1]:]]

    if crazy:
        crazy_loader += [DataLoader(torch.tensor(pfd[ind]).float(), batch_size=BATCH_SIZE, shuffle=False, num_workers=7, persistent_workers=True)]
    else:
        train_loader += [DataLoader(torch.tensor(train_pfd[ind]).float(), batch_size=BATCH_SIZE, shuffle=True, num_workers=7, persistent_workers=True)]
        val_loader += [DataLoader(torch.tensor(val_pfd[ind]).float(), batch_size=BATCH_SIZE, shuffle=True, num_workers=7, persistent_workers=True)]
        test_loader += [DataLoader(torch.tensor(test_pfd[ind]).float(), batch_size=BATCH_SIZE, shuffle=True, num_workers=7, persistent_workers=True)]

In [4]:
pfd = ProfileDataset(first_file)
print(pfd.__len__()) # first mode: 354166, second mode: 31087
print(len(pfd.__getitem__(0))) # should be 125 = 60 u + 60 rho + 4 TwTe, Me, omega_e, Re_d + 1 sigma

14237
125


In [4]:
########################## PROFILE AND VARIABLE NETWORK ##########################



class ProfileCNN_LatentFCN(nn.Module):
    def __init__(self, c_in=2, output=8, hidden_c=[4,8,4], alpha=0.1, kernels=[3, 2], hidden_l=[4+8,96,96,96,96,96,96,1]):
        super(ProfileCNN_LatentFCN, self).__init__()
        
        '''
        ProfileConv: maps an input of u, rho, Tw/Te, Re, omega, and Me into a predicted sigma value
        c_in - input channels of convolution, typically u and rho.  
        output - latent vector size, which encodes all relevant information from the boundary profile
        hidden_c - channels of all hidden convolutional and pooling layers.
        alpha - LeakyReLU coefficient
        kernels - kernels for convolution and for maxpooling.   
        '''
        self.convs = nn.ModuleList()
        self.nonlins = nn.ModuleList()
        self.maxpools = nn.ModuleList()

        self.channels = [c_in] + hidden_c
        self.hidden_l = hidden_l

        self.fcn = nn.ModuleList()
        self.nonlins2 = nn.ModuleList()

        # Define three convolutional, nonlin, and maxpool layers
        for i in range(len(self.channels)-1):
            self.convs.append(nn.Conv1d(in_channels=self.channels[i], out_channels=self.channels[i+1],
                               kernel_size=kernels[0]))
            self.nonlins.append(nn.LeakyReLU(alpha))
            self.maxpools.append(nn.MaxPool1d(kernel_size=kernels[1], stride=kernels[1]))

        # Then flatten the result and connect a linear layer to the output latent vectors
        self.flatten = nn.Flatten()
        self.latent = nn.Linear(4*5, output) # The final dimension will be 4x5 and we do fcn to 8 latent vars # OR 4 FOR 4,8,8,4?

        # Now we have 8 resultant latent parameters and 4 extra parameters that we pass through FCNs
        for i in range(len(hidden_l)-1):
            self.fcn.append(nn.Linear(hidden_l[i], hidden_l[i+1]))
            self.nonlins2.append(nn.LeakyReLU(alpha))

    def forward(self, x):
        '''
        Takes in input, containing u and rho as the first 60+60+4 params,
        pass it through a CNN, then pass the resultant 8-parameter latent vector
        and the four extra params into multiple FCNs to finally predict one value. 
        '''

        # First split up the data into u, rho, etc
        u = x[:, :60]
        rho = x[:, 60:120]
        u = u.unsqueeze(1)
        rho = rho.unsqueeze(1)
        y = torch.cat((u, rho), dim=1) # shape: (batch_size, 2, 60)
        
        TwTe = x[:, 120:121]
        Re = x[:, 121:122]
        omega = x[:, 122:123]
        mach = x[:, 123:124]
        periph = torch.cat((TwTe, Re, omega, mach), dim=1)

        # pass through conv layers
        for i in range(len(self.channels)-1):
            y = self.convs[i](y)
            y = self.nonlins[i](y)
            y = self.maxpools[i](y)

        # Now flatten and pass through fcn to get 8 latent variables
        y = self.flatten(y) # shape: (batch_size, 4 * 5) 
        y = self.latent(y) # shape: (batch_size, 8)

        y = torch.cat((y,periph), dim=1) # shape: (batch_size, 12)

        # pass through final fcns
        for i in range(len(self.hidden_l)-1):
            y = self.nonlins2[i](y)
            y = self.fcn[i](y)

        # final shape: (batch_size, 1)     
        return y
    

mod = ProfileCNN_LatentFCN()
mod.parameters()
ex = [[x for x in range(124)] for y in range(3)]
ex = torch.tensor(np.array(ex)).float()
print(ex.shape)
mod.forward(ex)
    

torch.Size([3, 124])


tensor([[-0.0272],
        [-0.0272],
        [-0.0272]], grad_fn=<AddmmBackward0>)

In [5]:
########################## AMPLIFICATION PREDICTOR ##########################



class AmplificationPredictor(L.LightningModule):
    def __init__(self, mode, **model_kwargs):
        super().__init__()
        self.save_hyperparameters()
        if mode==1:
            alpha = 0.1
        if mode==2:
            alpha = 0.2
        self.model = ProfileCNN_LatentFCN(alpha=alpha, **model_kwargs)
        #self.model = SimpleNN(**model_kwargs)

        
        self.split = 124
        #self.split = 3
        
    def forward(self, data):
        x = self.model(data[:, :self.split])  # Assuming model outputs predicted values
        return x

    def compute_loss_and_metric(self, batch):
        data, target = batch[:, :self.split], batch[:, self.split].float()  # Regression targets should be floats
        predictions = self.forward(data).squeeze()
        loss = torch.norm(predictions - target, p='fro') / torch.norm(target, p='fro') # Normalized root mean square error
        return loss

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=0.001, weight_decay=0.01)
        return optimizer

    def training_step(self, batch, batch_idx):
        loss = self.compute_loss_and_metric(batch)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.compute_loss_and_metric(batch)
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        loss = self.compute_loss_and_metric(batch)
        self.log("test_loss", loss)

In [6]:
########################### TRAINER CLASS ##########################



CHECKPOINT_PATH = "checkpoints\\"

def train_graph_classifier(model_name, train_loader, val_loader, test_loader, mode, **model_kwargs):
    L.seed_everything(42)

    # Create a PyTorch Lightning trainer with the generation callback
    root_dir = os.path.join(CHECKPOINT_PATH, model_name)
    os.makedirs(root_dir, exist_ok=True)
    trainer = L.Trainer(
        default_root_dir=root_dir,
        callbacks=[ModelCheckpoint(save_weights_only=True, mode="min", monitor="val_loss")],
        accelerator="auto",
        devices="auto",
        strategy="auto",
        max_epochs=200,
        enable_progress_bar=False,
    )
    trainer.logger._default_hp_metric = None

    # Check whether pretrained model exists. If yes, load it and skip training
    #pretrained_filename = os.path.join(CHECKPOINT_PATH, "%s.ckpt" % (model_name + str(mode)))
    L.seed_everything(42)
    model = AmplificationPredictor(mode=mode, **model_kwargs)
    trainer.fit(model, train_loader, val_loader)
    #model = AmplificationPredictor.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
    #trainer.save_checkpoint(pretrained_filename)

    # Test best model on validation and test set
    train_result = trainer.test(model, dataloaders=train_loader, verbose=False)
    test_result = trainer.test(model, dataloaders=test_loader, verbose=False)
    result = {"test": test_result[0]["test_loss"], "train": train_result[0]["test_loss"]}
    return model, result

In [7]:
########################## FIRST MODE TRAIN ##########################

model, result = train_graph_classifier(model_name="AmplificationPredictor",
                                    train_loader = train_loader[0],
                                    val_loader = val_loader[0],
                                    test_loader = test_loader[0],
                                    mode = 1)

# run this to make sure we have the right model 
print(" --- FIRST MODE ---")
print("Train performance: %4.2f%%" % (100.0 * result["train"]))
print("Test performance:  %4.2f%%" % (100.0 * result["test"]))


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\fouad\anaconda3\envs\gnn_env\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                 | Params | Mode 
-------------------------------------------------------
0 | model | ProfileCNN_LatentFCN | 48.3 K | train
-------------------------------------------------------
48.3 K    Trainable params
0         Non-trainable params
4

 --- FIRST MODE ---
Train performance: 1.29%
Test performance:  26.62%


In [7]:
########################## FIRST MODE SAVE/LOAD ##########################



try:
    if model.training:
        model.eval()
        print("Model set to eval mode.")
    # To save:
    torch.save(model, "first_mode.pth")
except:
    # To load:
    model = torch.load("first_mode.pth")

C:\Users\fouad\AppData\Local\Temp\ipykernel_25992\493736278.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("first_mode.pth")


In [17]:
########################## FIRST MODE PREDICTIONS #########################


allBLs = "allBLs"
root = "allBLs\\"

mach_dirs = [l for l in os.listdir(root)]
machs = [x*0.25 + 4 for x in range(13)]

P0 = 341700.0
gam = 1.4
T1 = 63.9231
R = 287.15
Tw = 300.0

# Now we want to loop through a list of frequencies and Reynolds numbers for each one of these profiles 
#freqs = [x*2500 + 20000 for x in range(100)] # 20,000 - 200,000 f
#freqs = [x*1000 + 20000 for x in range(100)]
#x_vals = [round(0.005*x + 0.001, 2) for x in range(100)] # 0.01 - 1.00 x
#x_vals = [round(0.004*x + 0.1, 3) for x in range(100)]

#PREDICTION!!!!

x_vals = [round(0.02*x + 0.1, 3) for x in range(100)]
freqs = [x*5000 + 20000 for x in range(100)]

res = []

for ind in range(len(mach_dirs)):
    m = mach_dirs[ind]
    mach = machs[ind]

    batch = []

    # find the bl profile
    _, bl_profile = bl_input_reader(root + m + "\\bl_input.dat", tpose=True)

    # Grab T_w/T_e which is the first value of T
    TwTe = bl_profile[4][0]
    Te = Tw/TwTe
    u_e = mach * np.sqrt(gam*R*Te)

    # We also want to figure out what the boundary thickness is. Since the length scale changes for each output in a frequency result,
    # we want to extract the boundary layer thickness that is nondimensionalized by lscl, which is the value of eta at which u converges 
    # to about 1. I will determine this value by identifying the point at which the difference between two eta values is < 1e-9 or so.
    eta = bl_profile[0]
    u_fine = bl_profile[2] # fine-grained u values to get accurate reading on boundary layer thickness
    ind = 0

    while(ind < len(u_fine)-1 and abs(u_fine[ind] - u_fine[ind+1]) > 1e-9): ind += 1
    thickness_eta = eta[ind]

    # Grab ~60 equidistant points of u and rho information from the boundary layer itself. We only care about the boundary layer so
    # ignore all points after 'ind'
    
    # to get exactly 60 points each time, first find how much buffer we need from the number of points on the boundary layer to get a 
    # number of points divisible by 60
    diff = (ind+1)%60
    # find where the new ind must be set; this number must be divisible by 60
    new_ind = ind + diff + 1 if (ind+diff+1)%60 == 0 else ind + 1 - diff
    step = round(new_ind/60) # just in case the boundary layer has less than 1 point?
    u = bl_profile[2][:new_ind:step]
    rho = bl_profile[1][:new_ind:step]
    

    arg = 1+0.5*(gam-1.)*mach**2.
    P1 = P0*(arg)**(-gam/(gam-1.))
    re  = P1/(R*T1)
    res_m = []
    mu_e = 1.45151376745308e-06*Te**1.5e0/(Te+110.4e0)

    for f in freqs:

        batch = []
        for x in x_vals: 

            # now we must determine our Re
            # first determine the lscl
            lscl = np.sqrt((mu_e*x)/(re*u_e))
            # determine thickness
            thickness = thickness_eta * lscl 
            # calculate Re_thickness
            Re = (re * u_e * thickness)/mu_e
            # calculate omega_thickness
            omega = (2*np.pi*f*thickness)/u_e
            
            # make input u, rho, TwTe, Re, omega, mach,
            point = []
            point += list(u) 
            point += list(rho) 
            point += [TwTe, Re, omega, mach]
            

            res += [point] # add to batch
        #res_m += [batch] 
    #res += [res_m]



In [9]:
########################## PREDICTIONS DATASET ##########################



class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32)

In [18]:
########################## PREPPING FIRST MODE PREDICTIONS ##########################



scaler2 = StandardScaler()
preds_set = CustomDataset(res)
preds_set = scaler2.fit_transform(preds_set)
pred_loader = DataLoader(preds_set, batch_size=200, shuffle=False)

predictions = []
with torch.no_grad():
    for batch in pred_loader:
        outputs = model(batch.float())
        predictions.extend(outputs.numpy()) 

predictions = torch.tensor(predictions)
res = torch.tensor(res)
sigs_inv = torch.tensor(sig_scaler[0].inverse_transform(predictions))

res = []
for ind in range(len(mach_dirs)):
    res += [(sigs_inv[10000*ind:10000*(ind+1)])]

final_res = []
for rm in res:
    ind = 0
    s1 = []
    for fi in range(len(freqs)):
        s2 = []
        for xi in range(len(freqs)):
            s2 += [rm[ind]]
            ind += 1
        s1 += [s2]
    final_res += [s1]
res = torch.tensor(final_res)
print(res.shape)


torch.Size([1, 100, 100])


In [ ]:
########################## GRAPHING FIRST MODE PREDICTIONS ##########################




tensor_np = res.numpy()

# Create contour plots for each slice
for i in range(tensor_np.shape[0]):
    extent = [x_vals[0], x_vals[len(x_vals)-1], freqs[0], freqs[len(freqs)-1]]

    plt.xlabel("x (m)")
    plt.ylabel("Frequency (f)")
    plt.title("Alpha Values, Mach " + str(0.25*i+4))
    plt.contourf(tensor_np[i], extent=extent)
    plt.colorbar()
    plt.savefig("first_mode_preds\\mach" + str(0.25*i+4) + ".pdf")
    plt.show()

In [18]:
######################### FIRST MODE GROUND TRUTH #########################



# Extract the positions of all important features from an example file
pos = all_pos("data\\constantRe.tar\\constantRe\\fp_M4.0_Tw_300K\\lst\\second_mode\\case_f_00100000\\nfact.dat")
sigma_pos = pos[0]
freq_pos = pos[1]
Re_pos = pos[2]
lscl_pos = pos[3]
ue_pos = pos[4]
mach = pos[5]
x_pos = pos[6]


root = "data\\constantRe.tar\\constantRe\\"
mach_dirs = [l for l in os.listdir(root)] # should be 4

mach_data = []

for j in range(len(mach_dirs)):

    m = j + 4
    batch = []

    # find the bl profile
    _, bl_profile = bl_input_reader(root + mach_dirs[j] + "\\mkBaseflow\\bl_input.dat", tpose=True)

    # We also want to figure out what the boundary thickness is. Since the length scale changes for each output in a frequency result,
    # we want to extract the boundary layer thickness that is nondimensionalized by lscl, which is the value of eta at which u converges 
    # to about 1. I will determine this value by identifying the point at which the difference between two eta values is < 1e-9 or so.
    eta = bl_profile[0]
    u_fine = bl_profile[2] # fine-grained u values to get accurate reading on boundary layer thickness
    ind = 0

    while(ind < len(u_fine)-1 and abs(u_fine[ind] - u_fine[ind+1]) > 1e-9): ind += 1
    thickness_eta = eta[ind]

    # Grab ~60 equidistant points of u and rho information from the boundary layer itself. We only care about the boundary layer so
    # ignore all points after 'ind'
    
    # to get exactly 60 points each time, first find how much buffer we need from the number of points on the boundary layer to get a 
    # number of points divisible by 60
    diff = (ind+1)%60
    # find where the new ind must be set; this number must be divisible by 60
    new_ind = ind + diff + 1 if (ind+diff+1)%60 == 0 else ind + 1 - diff
    step = round(new_ind/60) # just in case the boundary layer has less than 1 point?
    
    # Now find the relevant fmodes in this machdirectory
    lst_root = root + mach_dirs[j] + "\\lst\\first_mode\\"
    fmode_dirs = os.listdir(lst_root)
    fmode_dirs = reduce(lambda a,h: a+[h] if h[:7]=="case_f_" else a, fmode_dirs, []) # all fmode directories
    fmode = reduce(lambda a,h: a+[int(h[7:])] if h[:7]=="case_f_" else a, fmode_dirs, []) # all fmodes as ints

    data = []

    for f in range(len(fmode_dirs)): # search through all of the frequency directories 
        _, ndata, mach = ndata_reader(lst_root + fmode_dirs[f]+"\\nfact.dat")
        
        # Look through each line to craft a datapoint and then place it in the list. However, we only want data points
        # that give us x values from 0 to 1

        for line in range(len(ndata[0]) if len(ndata)>0 else 0):
            
            sig = ndata[sigma_pos][line]
            freq = ndata[freq_pos][line]
            lscl = ndata[lscl_pos][line]
            x = ndata[x_pos][line]

            # Require a dimensional value for boundary layer thickness. To do this, we take the nondim eta value, eta = y/lscl,
            # multiply it with lscl at this point, and have gotten a y value which defines the dim boundary layer thickness 
            thickness = thickness_eta * lscl

            # Now we re-dim Re_l and dim omega with boundary layer thickness 
            sig = sig * thickness_eta # thickness_eta = y/lscl, so sig * lscl * y/lscl = sig * y
            
            # add this to the data
            data += [(x, freq, sig)]

    mach_data += [data]

In [ ]:
######################### GRAPH FIRST MODE GROUND TRUTH #########################



stacked = reduce(lambda a,h: a + h, mach_data, [])
_, _, z = zip(*stacked)
vmin = min(z)
vmax = max(z)

m = 4
for data in mach_data:
    # Separate the data into x, y, and z components
    x, y, z = zip(*data)

    # Create a grid of x and y values
    xi = np.linspace(min(x), max(x), 100)
    yi = np.linspace(min(y), max(y), 100)
    xi, yi = np.meshgrid(xi, yi)

    # Interpolate the z values onto the grid
    zi = griddata((x, y), z, (xi, yi), method='linear')

    # Plot the contour plot
    plt.figure()
    cp = plt.contourf(xi, yi, zi, levels=14, cmap='viridis')
    plt.colorbar(cp)
    plt.xlabel('x (m)')
    plt.ylabel('Frequency (Hz)')
    plt.title('Alpha, Mach ' + str(m))
    plt.savefig("ground_truth\\first_mode\\mach" + str(m) + ".pdf")
    plt.show()
    m += 1

In [107]:
######################### TRAIN SECOND MODE AMP PREDICTOR #########################



model2, result = train_graph_classifier(model_name="AmplificationPredictor",
                                       train_loader = train_loader[1],
                                       val_loader = val_loader[1],
                                       test_loader = test_loader[1], 
                                       mode = 2)

print(" --- SECOND MODE ---")
print("Train performance: %4.2f%%" % (100.0 * result["train"]))
print("Test performance:  %4.2f%%" % (100.0 * result["test"]))

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                 | Params | Mode 
-------------------------------------------------------
0 | model | ProfileCNN_LatentFCN | 48.3 K | train
-------------------------------------------------------
48.3 K    Trainable params
0         Non-trainable params
48.3 K    Total params
0.193     Total estimated model params size (MB)
c:\Users\fouad\anaconda3\envs\gnn_env\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:475: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
c:\Users\fouad\anaconda3\envs\gnn_env\Lib\site-packages\lightning\pytorch\loops\fit_loop.py:298: The number of training batches (49) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower valu

 --- SECOND MODE ---
Train performance: 2.47%
Test performance:  72.38%


In [7]:
######################### SECOND MODE SAVE/LOAD #########################



try:
    if model2.training:
        model2.eval()
        print("Model set to eval mode.")

    torch.save(model2, "sec_mode.pth")
except:
    # To load:
    model2 = torch.load("sec_mode.pth")

C:\Users\fouad\AppData\Local\Temp\ipykernel_26596\197626645.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model2 = torch.load("sec_mode.pth")


In [45]:
######################### SECOND MODE PREDICTIONS #########################



allBLs = "allBLs"
root = "allBLs\\"

mach_dirs = [l for l in os.listdir(root)]
machs = [x*0.25 + 4 for x in range(13)]

P0 = 341700.0
gam = 1.4
T1 = 63.9231
R = 287.15
Tw = 300.0

# Now we want to loop through a list of frequencies and Reynolds numbers for each one of these profiles 
freqs = [x*5000 + 20000 for x in range(100)] # 20,000 - 200,000 f
x_vals = [round(0.005*x + 0.001, 2) for x in range(100)] # 0.01 - 1.00 x

x_vals = [round(0.02*x + 0.1, 3) for x in range(100)]
freqs = [x*5000 + 20000 for x in range(100)]

res = []

for ind in range(len(mach_dirs)):
    m = mach_dirs[ind]
    mach = machs[ind]

    batch = []

    # find the bl profile
    _, bl_profile = bl_input_reader(root + m + "\\bl_input.dat", tpose=True)

    # Grab T_w/T_e which is the first value of T
    TwTe = bl_profile[4][0]
    Te = Tw/TwTe
    u_e = mach * np.sqrt(gam*R*Te)

    # We also want to figure out what the boundary thickness is. Since the length scale changes for each output in a frequency result,
    # we want to extract the boundary layer thickness that is nondimensionalized by lscl, which is the value of eta at which u converges 
    # to about 1. I will determine this value by identifying the point at which the difference between two eta values is < 1e-9 or so.
    eta = bl_profile[0]
    u_fine = bl_profile[2] # fine-grained u values to get accurate reading on boundary layer thickness
    ind = 0

    while(ind < len(u_fine)-1 and abs(u_fine[ind] - u_fine[ind+1]) > 1e-9): ind += 1
    thickness_eta = eta[ind]

    # Grab ~60 equidistant points of u and rho information from the boundary layer itself. We only care about the boundary layer so
    # ignore all points after 'ind'
    
    # to get exactly 60 points each time, first find how much buffer we need from the number of points on the boundary layer to get a 
    # number of points divisible by 60
    diff = (ind+1)%60
    # find where the new ind must be set; this number must be divisible by 60
    new_ind = ind + diff + 1 if (ind+diff+1)%60 == 0 else ind + 1 - diff
    step = round(new_ind/60) # just in case the boundary layer has less than 1 point?
    u = bl_profile[2][:new_ind:step]
    rho = bl_profile[1][:new_ind:step]
    

    arg = 1+0.5*(gam-1.)*mach**2.
    P1 = P0*(arg)**(-gam/(gam-1.))
    re  = P1/(R*T1)
    res_m = []
    mu_e = 1.45151376745308e-06*Te**1.5e0/(Te+110.4e0)

    for f in freqs:

        batch = []
        for x in x_vals:

            # now we must determine our Re
            # first determine the lscl
            lscl = np.sqrt((mu_e*x)/(re*u_e))
            # determine thickness
            thickness = thickness_eta * lscl 
            # calculate Re_thickness
            Re = (re * u_e * thickness)/mu_e
            # calculate omega_thickness
            omega = (2*np.pi*f*thickness)/u_e
            
            # make input u, rho, TwTe, Re, omega, mach,
            point = []
            point += list(u) 
            point += list(rho) 
            point += [TwTe, Re, omega, mach]
            

            res += [point] # add to batch
    #res += [res_m]

In [46]:
########################## PREPPING SECOND MODE PREDICTIONS ##########################



scaler3 = StandardScaler()
preds_set = CustomDataset(res)
preds_set = scaler3.fit_transform(preds_set)
pred_loader = DataLoader(preds_set, batch_size=200, shuffle=False)

predictions = []
with torch.no_grad():
    for batch in pred_loader:
        outputs = model2(batch.float())
        predictions.extend(outputs.numpy()) 

predictions = torch.tensor(predictions)
res = torch.tensor(res)
sigs_inv = torch.tensor(sig_scaler[1].inverse_transform(predictions))

res = []
for ind in range(len(mach_dirs)):
    res += [(sigs_inv[10000*ind:10000*(ind+1)])]

final_res = []
for rm in res:
    ind = 0
    s1 = []
    for fi in range(len(freqs)):
        s2 = []
        for xi in range(len(freqs)):
            s2 += [rm[ind]]
            ind += 1
        s1 += [s2]
    final_res += [s1]
res = torch.tensor(final_res)
print(res.shape)


torch.Size([1, 100, 100])


In [ ]:
########################## GRAPHING SECOND MODE PREDICTIONS ##########################



vmin = res.min()
vmax = res.max()

tensor_np = res.numpy()

# Create contour plots for each slice
for i in range(tensor_np.shape[0]):
    extent = [0, x_vals[len(x_vals)-1], freqs[0], freqs[len(freqs)-1]]

    plt.xlabel("x (m)")
    plt.ylabel("Frequency (f)")
    plt.title("Alpha Values, Mach " + str(0.25*i + 4))
    plt.contourf(tensor_np[i], extent=extent)
    plt.colorbar()
    plt.savefig("sec_mode_preds\\mach" + str(0.25*i + 4) + ".pdf")
    plt.show()

In [21]:
######################### SECOND MODE GROUND TRUTH #########################

# Extract the positions of all important features from an example file
pos = all_pos("data\\constantRe.tar\\constantRe\\fp_M4.0_Tw_300K\\lst\\second_mode\\case_f_00100000\\nfact.dat")
sigma_pos = pos[0]
freq_pos = pos[1]
Re_pos = pos[2]
lscl_pos = pos[3]
ue_pos = pos[4]
mach = pos[5]
x_pos = pos[6]


root = "data\\constantRe.tar\\constantRe\\"
mach_dirs = [l for l in os.listdir(root)] # should be 4

mach_data = []

for j in range(len(mach_dirs)):

    m = ind + 4
    batch = []

    # find the bl profile
    _, bl_profile = bl_input_reader(root + mach_dirs[j] + "\\mkBaseflow\\bl_input.dat", tpose=True)

    # We also want to figure out what the boundary thickness is. Since the length scale changes for each output in a frequency result,
    # we want to extract the boundary layer thickness that is nondimensionalized by lscl, which is the value of eta at which u converges 
    # to about 1. I will determine this value by identifying the point at which the difference between two eta values is < 1e-9 or so.
    eta = bl_profile[0]
    u_fine = bl_profile[2] # fine-grained u values to get accurate reading on boundary layer thickness
    ind = 0

    while(ind < len(u_fine)-1 and abs(u_fine[ind] - u_fine[ind+1]) > 1e-9): ind += 1
    thickness_eta = eta[ind]

    # Grab ~60 equidistant points of u and rho information from the boundary layer itself. We only care about the boundary layer so
    # ignore all points after 'ind'
    
    # to get exactly 60 points each time, first find how much buffer we need from the number of points on the boundary layer to get a 
    # number of points divisible by 60
    diff = (ind+1)%60
    # find where the new ind must be set; this number must be divisible by 60
    new_ind = ind + diff + 1 if (ind+diff+1)%60 == 0 else ind + 1 - diff
    step = round(new_ind/60) # just in case the boundary layer has less than 1 point?
    
    # Now find the relevant fmodes in this machdirectory
    lst_root = root + mach_dirs[j] + "\\lst\\second_mode\\"
    fmode_dirs = os.listdir(lst_root)
    fmode_dirs = reduce(lambda a,h: a+[h] if h[:7]=="case_f_" else a, fmode_dirs, []) # all fmode directories
    fmode = reduce(lambda a,h: a+[int(h[7:])] if h[:7]=="case_f_" else a, fmode_dirs, []) # all fmodes as ints

    data = []

    for f in range(len(fmode_dirs)): # search through all of the frequency directories 
        _, ndata, mach = ndata_reader(lst_root + fmode_dirs[f]+"\\nfact.dat")
        
        # Look through each line to craft a datapoint and then place it in the list. However, we only want data points
        # that give us x values from 0 to 1

        for line in range(len(ndata[0]) if len(ndata)>0 else 0):
            
            sig = ndata[sigma_pos][line]
            freq = ndata[freq_pos][line]
            lscl = ndata[lscl_pos][line]
            x = ndata[x_pos][line]

            # Require a dimensional value for boundary layer thickness. To do this, we take the nondim eta value, eta = y/lscl,
            # multiply it with lscl at this point, and have gotten a y value which defines the dim boundary layer thickness 
            thickness = thickness_eta * lscl

            # Now we re-dim Re_l and dim omega with boundary layer thickness 
            sig = sig * thickness_eta # thickness_eta = y/lscl, so sig * lscl * y/lscl = sig * y
            
            # add this to the data
            data += [(x, freq, sig)]

    mach_data += [data]

In [ ]:
######################### GRAPH SECOND MODE GROUND TRUTH #########################

stacked = reduce(lambda a,h: a + h, mach_data, [])
_, _, z = zip(*stacked) 
vmin = min(z)
vmax = max(z)

m = 4
for data in mach_data:
    # Separate the data into x, y, and z components
    x, y, z = zip(*data)

    # Create a grid of x and y values
    xi = np.linspace(min(x), max(x), 100)
    yi = np.linspace(min(y), max(y), 100)
    xi, yi = np.meshgrid(xi, yi)

    # Interpolate the z values onto the grid
    zi = griddata((x, y), z, (xi, yi), method='linear')

    # Plot the contour plot
    plt.figure()
    cp = plt.contourf(xi, yi, zi, levels=14, cmap='viridis')
    plt.colorbar(cp)
    plt.xlabel('x (m)')
    plt.ylabel('Frequency (Hz)')
    plt.title('Alpha, Mach ' + str(m))
    plt.savefig("ground_truth\\sec_mode\\mach" + str(m) + ".pdf")
    plt.show()
    m += 1